<a href="https://colab.research.google.com/github/bghaendler/Data-Analysis/blob/master/Fraud_Detection_with_Synthetic_Financial_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fraud Detection with Synthetic Financial Datasets 

In [0]:
#data preprocessing
import pandas as pd
#math operations
import numpy as np


%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 1: Import Data from Kaggle

In [1]:
# Colab's file access feature
from google.colab import files

#retrieve uploaded file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 63 bytes


In [2]:
!kaggle datasets download ntnu-testimon/paysim1 --unzip 

 98% 178M/182M [00:01<00:00, 84.8MB/s]
100% 182M/182M [00:01<00:00, 107MB/s] 


* This dataset is presently only one of four on Kaggle with information on the **rising risk of digital financial fraud**, emphasizing the difficulty in obtaining such data. 
* The main technical challenge it poses to predicting fraud is the **highly imbalanced distribution between positive and negative classes** in 6 million rows of data. 
* Another stumbling block to the utility of this data stems from the **possible discrepancies in its description**. 
* The goal of this analysis is 
 * to solve both these issues by a detailed data exploration and cleaning followed by 
 * choosing a suitable machine-learning algorithm to deal with the skew. 
* I show that an optimal solution based on feature-engineering and e**xtreme gradient-boosted decision trees yields an enhanced predictive power of 0.997, as measured by the area under the precision-recall curve**. 
* Crucially, these results were obtained without artificial balancing of the data making this approach suitable to real-world applications.

# 2: Exploratory Data Analysis

In [0]:
pd.options.display.float_format = '${:,.2f}'.format

In [0]:
df_full = pd.read_csv('PS_20174392719_1491204439457_log.csv')
df_full = df_full.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})

In [82]:
df_full.sort_values("nameOrig").head(10)

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
3196942,249,CASH_OUT,"$244,486.46",C1000000639,"$8,946.00",$0.00,C785826240,"$526,950.37","$771,436.84",0,0
2798983,217,PAYMENT,"$3,170.28",C1000001337,"$58,089.00","$54,918.72",M216466820,$0.00,$0.00,0,0
1013585,46,PAYMENT,"$8,424.74",C1000001725,$783.00,$0.00,M1974356374,$0.00,$0.00,0,0
2980283,231,CASH_IN,"$261,877.19",C1000002591,"$7,596.00","$269,473.19",C503690069,"$1,126,627.70","$864,750.51",0,0
1920204,167,CASH_IN,"$20,528.65",C1000003372,"$2,302,074.12","$2,322,602.77",C1840417793,"$82,696.17","$62,167.52",0,0
3374567,254,CASH_IN,"$49,360.77",C1000003615,"$1,472,658.31","$1,522,019.07",C996088808,"$935,378.26","$886,017.50",0,0
4548211,327,CASH_OUT,"$211,189.64",C1000004053,$0.00,$0.00,C1128041097,"$2,609,840.34","$2,821,029.98",0,0
847860,41,CASH_OUT,"$93,865.13",C1000004530,"$351,422.72","$257,557.59",C1643839147,"$178,083.14","$271,948.26",0,0
2825292,226,TRANSFER,"$3,228,390.11",C1000005353,"$24,996.00",$0.00,C292963054,"$878,327.64","$4,106,717.75",0,0
2202461,185,CASH_OUT,"$233,109.79",C1000005555,"$20,623.00",$0.00,C1091802600,$0.00,"$233,109.79",0,0


In [86]:
df_full.isFraud.value_counts() 

0    6354407
1       8213
Name: isFraud, dtype: int64

In [87]:
df_full.isFlaggedFraud.value_counts() 

0    6362604
1         16
Name: isFlaggedFraud, dtype: int64

Descriptive Analysis of the DataFrame

In [88]:
df_full.describe()

,step,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
count,"$6,362,620.00","$6,362,620.00","$6,362,620.00","$6,362,620.00","$6,362,620.00","$6,362,620.00","$6,362,620.00","$6,362,620.00"
mean,$243.40,"$179,861.90","$833,883.10","$855,113.67","$1,100,701.67","$1,224,996.40",$0.00,$0.00
std,$142.33,"$603,858.23","$2,888,242.67","$2,924,048.50","$3,399,180.11","$3,674,128.94",$0.04,$0.00
min,$1.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
25%,$156.00,"$13,389.57",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
50%,$239.00,"$74,871.94","$14,208.00",$0.00,"$132,705.66","$214,661.44",$0.00,$0.00
75%,$335.00,"$208,721.48","$107,315.18","$144,258.41","$943,036.71","$1,111,909.25",$0.00,$0.00
max,$743.00,"$92,445,516.64","$59,585,040.37","$49,585,040.37","$356,015,889.35","$356,179,278.92",$1.00,$1.00


Test if there any missing values in DataFrame. It turns out there are no obvious missing values but, as we will see below, this does not rule out proxies by a numerical value like 0.

In [89]:
df_full.isnull().values.sum()

0

* In this section, we wrangle with the data exclusively using Dataframe methods. 
* This is the most succinct way to gain insights into the dataset. 
* More elaborate visualizations follow in subsequent sections.

## 2.1 Which types of transactions are fraudulent?
* We find that of the f**ive types of transactions**, *fraud occurs only in two of them* (see also kernels by Net, Philipp Schmidt and Ibe_Noriaki): 
 * '**TRANSFER**' where money is sent to a customer / fraudster and 
 *  '**CASH_OUT**' where money is sent to a merchant who pays the customer / fraudster in cash. 
* Remarkably, the number of fraudulent TRANSFERs almost equals the number of fraudulent CASH_OUTs (see the right half of the plot in section 5.1). 
* These observations appear, at first, to bear out the description provided on Kaggle for the modus operandi of fraudulent transactions in this dataset, namely, fraud is committed by first transferring out funds to another account which subsequently cashes it out. 
* We will return to this issue later in section 2.4


In [90]:
print('The types of fraudulent transactions are {}'.format(\
list(df_full.loc[df_full.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' 
                                                             # & 'TRANSFER'

dfFraudTransfer = df_full.loc[(df_full.isFraud == 1) & (df_full.type == 'TRANSFER')]
dfFraudCashout = df_full.loc[(df_full.isFraud == 1) & (df_full.type == 'CASH_OUT')]

print ('The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) # 4097
print ('The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116

The types of fraudulent transactions are ['TRANSFER', 'CASH_OUT']
The number of fraudulent TRANSFERs = 4097
The number of fraudulent CASH_OUTs = 4116


## 2.2. What determines whether the feature isFlaggedFraud gets set or not?
* It turns out that **the origin of isFlaggedFraud is unclear**, contrasting with the description provided. 
 * The 16 entries (out of 6 million) where the isFlaggedFraud feature is set **do not seem to correlate with any explanatory variable**. 
 * The data is described as isFlaggedFraud being set when an attempt **is made to 'TRANSFER' an 'amount' greater than 200,000**. 
   * In fact, as shown below, isFlaggedFraud can remain not set despite this condition being met.

In [91]:
print('The type of transactions in which isFlaggedFraud is set: \
{}'.format(list(df_full.loc[df_full.isFlaggedFraud == 1].type.drop_duplicates()))) 
                                                            # only 'TRANSFER'

dfTransfer = df_full.loc[df_full.type == 'TRANSFER']
dfFlagged = df_full.loc[df_full.isFlaggedFraud == 1]
dfNotFlagged = df_full.loc[df_full.isFlaggedFraud == 0]

print('Min amount transacted when isFlaggedFraud is set= {}'\
                                  .format(dfFlagged.amount.min())) # 353874.22
print('Max amount transacted when isFlaggedFraud is set= {}'\
                                  .format(dfFlagged.amount.max())) # 353874.22
print('Mean amount transacted when isFlaggedFraud is set= {}'\
                                  .format(dfFlagged.amount.mean())) # 353874.22
print('Max amount transacted in a TRANSFER where isFlaggedFraud is not set=\
 {}'.format(dfTransfer.loc[dfTransfer.isFlaggedFraud == 0].amount.max())) # 92445516.64
print('Mean amount transacted in a TRANSFER where isFlaggedFraud is not set=\
 {}'.format(dfTransfer.loc[dfTransfer.isFlaggedFraud == 0].amount.mean())) # 92445516.64

The type of transactions in which isFlaggedFraud is set: ['TRANSFER']
Min amount transacted when isFlaggedFraud is set= 353874.22
Max amount transacted when isFlaggedFraud is set= 10000000.0
Mean amount transacted when isFlaggedFraud is set= 4861597.730625
Max amount transacted in a TRANSFER where isFlaggedFraud is not set= 92445516.64
Mean amount transacted in a TRANSFER where isFlaggedFraud is not set= 910528.3831828752


* **Can oldBalanceDest and newBalanceDest determine isFlaggedFraud being set? **
 * The old is identical to the new balance in the origin and destination accounts, for every TRANSFER where isFlaggedFraud is set. 
 * This is **presumably because the transaction is halted** [4]. 
 * Interestingly, oldBalanceDest = 0 in every such transaction. 
 * However, as shown below, since isFlaggedFraud can remain not set in TRANSFERS where oldBalanceDest and newBalanceDest can both be 0, these conditions do not determine the state of isFlaggedFraud.

In [93]:
dfTransfer.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldBalanceOrig',
       'newBalanceOrig', 'nameDest', 'oldBalanceDest', 'newBalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [94]:
print('\nThe number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and\
 newBalanceDest = 0: {}'.\
      format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceDest == 0) & (dfTransfer.newBalanceOrig == 0)])))


The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0: 60821


* isFlaggedFraud being set cannot be thresholded on oldBalanceOrig since the corresponding range of values overlaps with that for TRANSFERs where isFlaggedFraud is not set (see below). 
* Note that we do not need to consider newBalanceOrig since it is updated only after the transaction, whereas isFlaggedFraud would be set before the transaction takes place.

In [96]:
print('Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: {}'.\
format([round(dfFlagged.oldBalanceOrig.min()), round(dfFlagged.oldBalanceOrig.max())]))

print('Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where \
oldBalanceOrig = \
newBalanceOrig: {}'.format(\
[dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
== dfTransfer.newBalanceOrig)].oldBalanceOrig.min(), \
round(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
               == dfTransfer.newBalanceOrig)].oldBalanceOrig.max())]))

Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: [353874.0, 19585040.0]
Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: [0.0, 575668.0]


* Can isFlaggedFraud be set based on seeing a customer transacting more than once? 
 * Note that duplicate customer names don't exist within transactions where isFlaggedFraud is set, but duplicate customer names exist within transactions where isFlaggedFraud is not set. 
 * It turns out that originators of transactions that have isFlaggedFraud set have transacted only once. 
 * Very few destination accounts of transactions that have isFlaggedFraud set have transacted more than once.





In [99]:
print('Have originators of transactions flagged as fraud transacted more than \
once? {}'\
.format((dfFlagged.nameOrig.isin(pd.concat([dfNotFlagged.nameOrig, \
                                    dfNotFlagged.nameDest]))).any())) # False

print('Have destinations for transactions flagged as fraud initiated\
 other transactions? \
{}'.format((dfFlagged.nameDest.isin(dfNotFlagged.nameOrig)).any())) # False

# Since only 2 destination accounts of 16 that have 'isFlaggedFraud' set have been
# destination accounts more than once,
# clearly 'isFlaggedFraud' being set is independent of whether a 
# destination account has been used before or not

print('How many destination accounts of transactions flagged as fraud have been \
destination accounts more than once?: {}'\
.format(sum(dfFlagged.nameDest.isin(dfNotFlagged.nameDest)))) # 2

Have originators of transactions flagged as fraud transacted more than once? False
Have destinations for transactions flagged as fraud initiated other transactions? False
How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 2


* It can be easily seen that transactions with isFlaggedFraud set occur at all values of step, similar to the complementary set of transactions. 
* Thus isFlaggedFraud does not correlate with step either and is therefore seemingly unrelated to any explanatory variable or feature in the data

**Conclusion:** 
* Although isFraud is always set when isFlaggedFraud is set, since isFlaggedFraud is set just 16 times in a seemingly meaningless way, **we can treat this feature as insignificant and discard it in the dataset without loosing information.**

## 2.3. Are expected merchant accounts accordingly labelled?

* It was stated [5] that CASH_IN involves being paid by a merchant (whose name is prefixed by 'M'). 
 * However, as shown below, the present data does not have merchants making CASH_IN transactions to customers.

In [100]:
print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(df_full.loc[df_full.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False


Are there any merchants among originator accounts for CASH_IN transactions? False


* Similarly, it was stated that CASH_OUT involves paying a merchant. However, for CASH_OUT transactions there are no merchants among the destination accounts.

In [103]:
print('Are there any merchants among destination accounts for CASH_OUT \
transactions? {}'.format(\
(df_full.loc[df_full.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False

Are there any merchants among destination accounts for CASH_OUT transactions? False


* In fact, there are no merchants among any originator accounts. Merchants are only present in destination accounts for all PAYMENTS.

In [104]:
print('Are there merchants among any originator accounts? {}'.format(\
      df_full.nameOrig.str.contains('M').any())) # False

print('Are there any transactions having merchants among destination accounts\
 other than the PAYMENT type? {}'.format(\
(df_full.loc[df_full.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False

Are there merchants among any originator accounts? False
Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


* Conclusion: Among the account labels nameOrig and nameDest, for all transactions, the merchant prefix of 'M' occurs in an unexpected way.

## 2.4. Are there account labels common to fraudulent TRANSFERs and CASH_OUTs?
* From the data description, the modus operandi for committing fraud involves first making a TRANSFER to a (fraudulent) account which in turn conducts a CASH_OUT. 
* CASH_OUT involves transacting with a merchant who pays out cash. 
* Thus, within this two-step process, **the fraudulent account would be both**, the destination in a TRANSFER and the originator in a CASH_OUT. 
* However, the data shows below that there are no such common accounts among fraudulent transactions. 
* Thus, the data is not imprinted with the expected modus-operandi.

In [105]:
print('Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? {}'.format(\
(dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False
dfNotFraud = df_full.loc[df_full.isFraud == 0]

Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


* Could destination accounts for fraudulent TRANSFERs originate CASHOUTs that are not detected and are labeled as genuine? It turns out there are 3 such accounts.

In [106]:
print('\nFraudulent TRANSFERs whose destination accounts are originators of \
genuine CASH_OUTs: \n\n{}'.format(dfFraudTransfer.loc[dfFraudTransfer.nameDest.\
isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]))


Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

         step      type        amount  ... newBalanceDest  isFraud  isFlaggedFraud
1030443    65  TRANSFER $1,282,971.57  ...          $0.00        1               0
6039814   486  TRANSFER   $214,793.32  ...          $0.00        1               0
6362556   738  TRANSFER   $814,689.88  ...          $0.00        1               0

[3 rows x 11 columns]


In [109]:
dfFraudTransfer.loc[dfFraudTransfer.nameDest.isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
1030443,65,TRANSFER,"$1,282,971.57",C1175896731,"$1,282,971.57",$0.00,C1714931087,$0.00,$0.00,1,0
6039814,486,TRANSFER,"$214,793.32",C2140495649,"$214,793.32",$0.00,C423543548,$0.00,$0.00,1,0
6362556,738,TRANSFER,"$814,689.88",C2029041842,"$814,689.88",$0.00,C1023330867,$0.00,$0.00,1,0


* However, 2 out of 3 of these accounts first make a genuine CASH_OUT and only later (as evidenced by the time step) receive a fraudulent TRANSFER. 
**Thus, fraudulent transactions are not indicated by the nameOrig and nameDest features.**

**Conclusion: **
* Noting from section 2.3 above that the nameOrig and nameDest features neither encode merchant accounts in the expected way, below, we drop these features from the data since they are meaningless.

# 3: Data cleaning

* From the exploratory data analysis (EDA) of section 2, we know that **fraud only occurs in 'TRANSFER's and 'CASH_OUT's**. 
So we assemble only the corresponding data in X for analysis.

In [0]:
X = df_full.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]

randomState = 5
np.random.seed(randomState)

#X = X.loc[np.random.choice(X.index, 100000, replace = False)]

Y = X['isFraud']
del X['isFraud']

# Eliminate columns shown to be irrelevant for analysis in the EDA
X = X.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)

# Binary-encoding of labelled data in 'type'
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int) # convert dtype('O') to dtype(int)

In [114]:
X.head(10)

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
2,1,0,$181.00,$181.00,$0.00,$0.00,$0.00
3,1,1,$181.00,$181.00,$0.00,"$21,182.00",$0.00
15,1,1,"$229,133.94","$15,325.00",$0.00,"$5,083.00","$51,513.44"
19,1,0,"$215,310.30",$705.00,$0.00,"$22,425.00",$0.00
24,1,0,"$311,685.89","$10,835.00",$0.00,"$6,267.00","$2,719,172.89"
42,1,1,"$110,414.71","$26,845.41",$0.00,"$288,800.00","$2,415.16"
47,1,1,"$56,953.90","$1,942.02",$0.00,"$70,253.00","$64,106.18"
48,1,1,"$5,346.89",$0.00,$0.00,"$652,637.00","$6,453,430.91"
51,1,1,"$23,261.30","$20,411.53",$0.00,"$25,742.00",$0.00
58,1,0,"$62,610.80","$79,114.00","$16,503.20",$517.00,"$8,383.29"


In [113]:
Y.head(10)

2     1
3     1
15    0
19    0
24    0
42    0
47    0
48    0
51    0
58    0
Name: isFraud, dtype: int64

## 3.1. Imputation of Latent Missing Values
* The data has several transactions with zero balances in the destination account both before and after a non-zero amount is transacted. 
* The fraction of such transactions, where zero likely **denotes a missing value, is much larger in fraudulent (50%) compared to genuine transactions (0.06%).**

In [116]:
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]
print('The fraction of fraudulent transactions with \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceDest == 0) & \
(Xfraud.newBalanceDest == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud))))

print('The fraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceDest == 0) & \
(XnonFraud.newBalanceDest == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud))))

The fraction of fraudulent transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.4955558261293072
The fraction of genuine transactions with 'oldBalanceDest' = newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0006176245277308345


In [118]:
XnonFraud.head(10)

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
15,1,1,"$229,133.94","$15,325.00",$0.00,"$5,083.00","$51,513.44"
19,1,0,"$215,310.30",$705.00,$0.00,"$22,425.00",$0.00
24,1,0,"$311,685.89","$10,835.00",$0.00,"$6,267.00","$2,719,172.89"
42,1,1,"$110,414.71","$26,845.41",$0.00,"$288,800.00","$2,415.16"
47,1,1,"$56,953.90","$1,942.02",$0.00,"$70,253.00","$64,106.18"
48,1,1,"$5,346.89",$0.00,$0.00,"$652,637.00","$6,453,430.91"
51,1,1,"$23,261.30","$20,411.53",$0.00,"$25,742.00",$0.00
58,1,0,"$62,610.80","$79,114.00","$16,503.20",$517.00,"$8,383.29"
60,1,1,"$82,940.31","$3,017.87",$0.00,"$132,372.00","$49,864.36"
70,1,1,"$47,458.86","$209,534.84","$162,075.98","$52,120.00",$0.00


In [117]:
Xfraud.head(10)

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
2,1,0,$181.00,$181.00,$0.00,$0.00,$0.00
3,1,1,$181.00,$181.00,$0.00,"$21,182.00",$0.00
251,1,0,"$2,806.00","$2,806.00",$0.00,$0.00,$0.00
252,1,1,"$2,806.00","$2,806.00",$0.00,"$26,202.00",$0.00
680,1,0,"$20,128.00","$20,128.00",$0.00,$0.00,$0.00
681,1,1,"$20,128.00","$20,128.00",$0.00,"$6,268.00","$12,145.85"
724,1,1,"$416,001.33",$0.00,$0.00,$102.00,"$9,291,619.62"
969,1,0,"$1,277,212.77","$1,277,212.77",$0.00,$0.00,$0.00
970,1,1,"$1,277,212.77","$1,277,212.77",$0.00,$0.00,"$2,444,985.19"
1115,1,0,"$35,063.63","$35,063.63",$0.00,$0.00,$0.00


In [0]:
X.loc[(X.oldBalanceDest == 0) & (X.newBalanceDest == 0) & (X.amount != 0), \
      ['oldBalanceDest', 'newBalanceDest']] = - 1

In [120]:
X.head()

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
2,1,0,$181.00,$181.00,$0.00,$-1.00,$-1.00
3,1,1,$181.00,$181.00,$0.00,"$21,182.00",$0.00
15,1,1,"$229,133.94","$15,325.00",$0.00,"$5,083.00","$51,513.44"
19,1,0,"$215,310.30",$705.00,$0.00,"$22,425.00",$0.00
24,1,0,"$311,685.89","$10,835.00",$0.00,"$6,267.00","$2,719,172.89"


In [0]:
The data also has several transactions with zero balances in the originating account both before and after a non-zero amount is transacted. In this case, the fraction of such transactions is much smaller in fraudulent (0.3%) compared to genuine transactions (47%). Once again, from similar reasoning as above, instead of imputing a numerical value we replace the value of 0 with a null value.

* The data also has several transactions with zero balances in the originating account both before and after a non-zero amount is transacted. 
* In this case, the fraction of such transactions is much smaller in fraudulent (0.3%) compared to genuine transactions (47%). 
* Once again, from similar reasoning as above, instead of imputing a numerical value we replace the value of 0 with a null value.